# Analysis of Physical Oceanographic Data - SIO 221A
### Python version of [Sarah Gille's](http://pordlabs.ucsd.edu/sgille/sioc221a/index.html) notes by:
#### Bia Villas Bôas (avillasboas@ucsd.edu) & Gui Castelão (castelao@ucsd.edu)

## Lecture 14

*Reading:  Bendat and Piersol, Ch. 5.1-5.2, with attention to cross-covariance and cross-spectrum*

#### Covariance

Early in the quarter we discussed the variance, and we left for later
the concept of correlation or covariance.  If we want to compare
two time series, we can compute the variance of one record relative to
the other.  Formally we can write:

\begin{equation}
\text{cov}({x,y}) = \langle x(t) y(t)\rangle. \hspace{3cm} (1)
\end{equation}

or in discrete terms

\begin{equation}
\text{cov}({x,y}) = \frac{1}{N}\sum_{i=1}^N x_i y_i. \hspace{3cm} (2)
\end{equation}

For comparison purposes, we often normalize this to produce a
correlation coefficient, which is normalized by the variance:

\begin{equation}
r = \frac{\frac{1}{N}\sum_{i=1}^N x_i y_i}{\sqrt{\frac{1}{N} \sum_{i=1}^N x_i^2
\frac{1}{N} \sum_{i=1}^N y_i^2}}. \hspace{3cm} (3)
\end{equation}

(You might wonder how to judge whether a correlation coefficient is
statistically significant.  Correlation coefficients should have a Gaussian
distribution, which means that cumulative distribution function will be
an error function.  We can use this to determine the correlation coefficient
that we might expect from an equivalent number of random white noise
variables:

\begin{equation}
\delta r = \text{erf}^{-1}(p) \sqrt{\frac{2}{N}} \hspace{3cm} (4)
\end{equation}

where $p$ is the significance level we want to consider, typically 0.95,
and $N$ is the effective number of degrees of freedom.)

#### Coherence

Coherence provides information that is analagous to a correlation
coefficient for Fourier transforms. It tells us whether two series
are statistically
linked at any specific frequency.  This can be important if we think that
the records are noisy or otherwise uncorrelated at some frequencies, but
that they also contain statistically correlated signals.

To compute coherence, first we need a cross-spectrum.  (We looked
at this in passing when we considered Parseval's theorem, but at that
stage, I quickly set my different variables equal to each other.)  Consider
two time series $x(t)$ and $y(t)$:

\begin{eqnarray}
x(t) & = & \sum_{n=-\infty}^{\infty} X_n e^{i2\pi f_n t} \hspace{3cm} (5)\\
y(t) & = & \sum_{n=-\infty}^{\infty} Y_n e^{i2\pi f_n t} \hspace{3cm} (6)
\end{eqnarray}

The the cross spectrum is computed in analogy with the spectrum:

\begin{equation}
\hat{S}_{XY}(f_m)= \frac{\langle X_m^* Y_m\rangle}{T}\hspace{3cm} (7)
\end{equation}

The relationship between the cross-spectrum and the covariance is analogous
to the relationship between the spectrum and the variance.  There are
some important details to notice.

1. The cross spectrum is complex, while the spectrum was real.
2. The cross spectrum is computed as an average of multiple spectral segments.
3. In our discrete Fourier transform, we should be normalizing by $N$, as always, but we're mostly concerned with relative values.

The cross-spectrum is complex, and when we use it we distinguish between
the real and imaginary parts.
The real part is called the "*co-spectrum*":

\begin{equation}
C(f_k) = \frac{1}{N} \Re{\sum_{n=1}^N(X_k Y_k^*)} \hspace{3cm} (8)
\end{equation}
and the imaginary part is called the "*quadrature spectrum*"

\begin{equation}
Q(f_k) = \frac{1}{N} \Im{\sum_{n=1}^N(X_k Y_k^*)}. \hspace{3cm} (9)
\end{equation}

To determine the frequency-space relationship between two data sets $x_n$ and
$y_n$, we first divide them into segments and Fourier transform them, so that
we have a set of $X_k$'s and a set of $Y_k$'s.  When we computed spectra,
we found the amplitude of each $X_k$ and then summed over all our segments.
Now we're going to do something slightly different.  For each segment pair,
we'll compute the product of $X$ times the complex conjugate of $Y$:
$X_k Y_k^*$.  Then we'll sum over all the segments.  In Python this becomes

```python
np.sum(X*Y.conjugate(), axis=0)
````
The corresponding amplitude is $\sqrt{C^2(f_k) + Q^2(f_k)}$.
For comparison the spectra for $X$ was:

\begin{equation}
S_{xx}(f_k) = \frac{1}{N} \sum_{n=1}^N X_k X_k^*, \hspace{3cm} (10)
\end{equation}

and it was always real.

The coherence resembles a correlation coefficient.  It's the amplitude squared
divided by the power spectral amplitudes for each of the two components:

\begin{equation}
\gamma_{xy}^2(f_k) = \frac{C^2(f_k) + Q^2(f_k)}{S_{xx}(f_k) S_{yy}(f_k)}.\hspace{3cm} (11)
\end{equation}

(Sometimes you'll see $G_{xx}$, $G_{yy}$, and $G_{xy}$ in place of $S_{xx}$,
$S_{yy}$, and $S_{xy}$.  Bendat and Piersol define $S$ to represent the two
sided cross-spectra density and $G$ to represent to represent one-sided spectra.)

In addition to the coherence amplitude, we can also infer a phase.
The phase $\phi(f_k) = \tan^{-1}(-Q(f_k)/C(f_k))$
tells us the timing difference between the two time series.  If $\phi = 0$,
changes in $x$ and $y$ happen at the same time.  If $\phi = \pi$, then
$x$ is at a peak when $y$ is at a trough.  And a value of $\phi=\pi/2$ or
$\phi=-\pi/2$ tells us that the records are a quarter cycle different.

#### Digression:  Extracting phase information from the Fourier coefficients

After all this effort to square Fourier coefficients, you might wonder
what the real and imaginary parts are really good for.  They are useful
for sorting out the phasing of your sinusoidal oscillations.  When is
the amplitude at a maximum?  To do this you can keep in mind that

\begin{equation}
A\cos(\sigma t + \phi) = a\cos(\sigma t) + b\sin(\sigma t). \hspace{3cm} (12)
\end{equation}

This can be rewritten:

\begin{equation}
\cos(\sigma t)\cos(\phi) - \sin(\sigma t)\sin(\phi) = \frac{a}{A}\cos(\sigma t) + \frac{b}{A}\sin(\sigma t), \hspace{3cm} (13)
\end{equation}

which means that

\begin{eqnarray}
\frac{a}{A} & = & \cos(\phi) \hspace{3cm} (14)\\
\frac{b}{A} & = & -\sin(\phi) \hspace{3cm} (15)
\end{eqnarray}

so

\begin{equation}
\phi=\mbox{atan}\left(-\frac{b}{a}\right). \hspace{3cm} (16)
\end{equation}

Actually there's more information in the Fourier coefficients than this
conveys, since you know the signs of both $a$ and $b$, and not just their
relative magnitudes. The arctangent function doesn't distinguish +45\degrees
from -135\degrees, but we can.  In some numerical implementations, you can
address this using a function called `arctan2`.

```python
phi = np.arcatan2(-b, a)
```

#### Coherence:  Hypothetical Example for Mission Bay Channel

The power of coherence comes because it gives us a means to compare two
different variables.  With spectra we can ask, is there energy at a given
frequency?  With coherence we can ask whether wind energy at a given
frequency drives an ocean response at a given frequency.  Does the ocean
respond to buoyancy forcing?  Does momentum vary with wind?  Does
one geographic location vary with another location?  Coherence is our
window into the underlying physics of the system.

Let's put this to work, starting with an idealized case:
Suppose we want to estimate currents entering and leaving the Mission
Bay Channel.  How do waves travel through the channel?  We can represent
this with a dispersion relationship describing the dominant propagation
in frequency-wavenumber space:  $k=K(f)$.

You could imagine measuring Mission Bay by installing one current meter
(with a cost of \\$10-\\$20,000), but another approach is to install a couple
of pressure
recorders along the axis of the channel (at a cost of \\$1000 each).
Let's assume all waves come from the ocean, and travel along
the channel axis at speed $V=c+U_{current}$, where $c$ is the wave speed
and $U_{current}$ the background current speed.  If the waves are
surface gravity waves, $c=\sqrt{gD}$.  The sensors measure time series
of pressure only, so provide frequency information $f$.  How does
$f$ relate to velocity?  If we have a wavenumber $2\pi k=2\pi/\lambda$,
what does the pressure sensor see?  It will detect frequencies $f=kV =
k(c+U_{current})$.  So we can compute the cross spectrum between our
two records.

Let's test this out.  We'll define a hypothetical data set:

In [ ]:
import numpy as np
from numpy import pi, cos, sin
import matplotlib.pyplot as plt

np.random.seed(0)

N = 5000
lamb = 10 #10 m wavelength
V = 0.3 #0.3 m/s propagation
n2s = 0.2 #noise-to-signal ratio
time = np.arange(N)
x = n2s*np.random.randn(N) + cos(2*pi/lamb*V*time)
y = n2s*np.random.randn(N) + cos(2*pi/lamb*V*time + pi/2)

fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(12, 8))
ax1.plot(time, x)
ax1.plot(time, y)
ax2.plot(time[:50], x[:50])
ax2.plot(time[:50], y[:50])

What happens if you Fourier transform without bothering to segment?
Then the data end up being unrevealing. We can demonstrate this:

In [ ]:
fx = np.fft.fft(x)
fy = np.fft.fft(y)
sx = abs(fx[:N//2+1])**2
sx[1:-1] = 2*sx[1:-1]

sy = abs(fy[:N//2+1])**2
sy[1:-1] = 2*sy[1:-1]

cxy = fx[:N//2+1].conjugate()*fy[:N//2+1] 
cxy[1:-1] = 2*cxy[1:-1]
C = abs(cxy)/np.sqrt(sx*sy)

In [ ]:
plt.plot(C)

In this case, the coherence is 1 everywhere.  Why is that?
Because without averaging, we're merely computing:

\begin{eqnarray}
\gamma_{xy}^2 & = & \frac{(X^*Y)^* (X^*Y)}{X^*X Y^*Y} \hspace{3cm} (17)\\
       & = & \frac{XY^*X^*Y}{X^*X Y^*Y} \hspace{3cm} (18)\\
       & = & \frac{X^*X Y^*Y}{X^*X Y^*Y} \hspace{3cm} (19)\\
       & = & 1 \hspace{3cm} (20)
\end{eqnarray}

When it's done properly,
coherence measures how well different segments of $x$ and $y$
show the same type of relationship at a given frequency.
We need the averaging to find out if the phase relationship between
$x$ and $y$ is repeatable.  With only one segment, both $x$ and $y$
are guaranteed to have information at each frequency with a definable
phase relationship between $x$ and $y$.  The multiple segments allow
us to test whether this phase relationship is relatively stable in
time:  does $x$ always lead $y$ by about the same fraction of a cycle?

When it's done properly,
coherence measures how well different segments of $x$ and $y$
show the same type of relationship at a given frequency.
We need the averaging to find out if the phase relationship between
$x$ and $y$ is repeatable.  With only one segment, both $x$ and $y$
are guaranteed to have information at each frequency with a definable
phase relationship between $x$ and $y$.  The multiple segments allow
us to test whether this phase relationship is relatively stable in
time:  does $x$ always lead $y$ by about the same fraction of a cycle?

To do the coherence calculation more constructively, we determine the frequency-space relationship between two data sets $x_n$ and
$y_n$, by first dividing them into segments and then Fourier transforming
them, so that
we have a set of $X_k$'s and a set of $Y_k$'s.  When we computed spectra,
we found the amplitude of each $X_k$ and then summed over all our segments.
Now we're going to do something slightly different.  For each segment pair,
we'll compute the product of $X$ times the complex conjugate of $Y$:
$X_k Y_k^*$.  Then we'll sum over all the segments.
In Python this becomes:

In [ ]:
import sys
sys.path.append('../../tools/')

from segment import overlapping_segments1D 

In [ ]:
segment_length = 500
x_use = overlapping_segments1D(x, segment_length, overlap=0.5)
y_use = overlapping_segments1D(y, segment_length, overlap=0.5)
M, p = y_use.shape
print(x_use.shape)
print(y_use.shape)

In [ ]:
fx = np.fft.fft(x_use, axis=-1) #should window and detrend here, but we're
#skipping that for now
fy = np.fft.fft(y_use, axis=-1)

sx = np.sum(abs(fx[:, :p//2+1]/p)**2, axis=0)
sx[1:-1] = 2*sx[1:-1]

sy = np.sum(abs(fy[:, :p//2+1]/p)**2, axis=0)
sy[1:-1] = 2*sy[1:-1]

cxy = np.sum(fx[:, :p//2+1].conjugate()*fy[:, :p//2+1]/p/p, axis=0) 
cxy[1:-1] = 2*cxy[1:-1]
C = abs(cxy)/np.sqrt(sx*sy)

phase_C = np.arctan2(-cxy.imag, cxy.real)

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 6))
ax1.plot(C)
ax1.set_ylabel('Coherence', fontsize=14)
ax2.plot(phase_C*180/pi)
ax2.set_ylabel('Phase [degrees]', fontsize=14)

The phase difference that emerges from this is only relevant at the phase
where there is coherence energy (15 cycles/1000 points in the example above),
and in that case the phase is a quarter cycle different, with relatively
small error bars.  If we reverse the
order of $x$ and $y$, we'll find negative phase, so a lead will turn into a
lag.
`

The phase $\phi(f_k) = \tan^{-1}(-Q(f_k)/C(f_k))$
tells us the timing difference between the two time series.  If $\phi = 0$,
changes in $x$ and $y$ happen at the same time.  If $\phi = \pi$, then
$x$ is at a peak when $y$ is at a trough.  And a value of $\phi=\pi/2$ or
$\phi=-\pi/2$ tells us that the records are a quarter cycle different.

#### Example:  Coherence and Wave Spectra

So let's see whether surfboard acceleration measurements show any
signs of coherence.
We'll start by comparing vertical and horizontal accelerations of
the free floating accelerometer, as shown in Figure 1.

<img src="files/lec14f1.png" alt="fig1" width="500"/>

Figure 1: *Time series of vertical acceleration and x-axis accleration for free-floating accelerometer near
Scripps pier.*

These two records have rather different spectra as shown in
Figure 2.

<img src="files/lec14f2.png" alt="fig1" width="500"/>

Figure 2: *Spectra for vertical and $x$ acceleration of free-floating
accelerometer near Scripps pier.*


The two records are coherent, as shown in Figure 3.
with a phase difference of roughly $\pi$ radians, implying that they are
180$^\circ$ out of phase, at least at the frequencies at which they are
actually coherent.

<img src="files/lec14f3.png" alt="fig1" width="500"/>

Figure 3: *(top) Coherence of vertical and $x$ acceleration of free-floating
accelerometer near Scripps pier.
(bottom) Phase difference between vertical and $x$ acceleration components.*

         
In contrast, the vertical acceleration for the free floating accelerometer
is not coherent with vertical acceleration from the shortboard.

<img src="files/lec14f4.png" alt="fig1" width="500"/>

Figure 4: *(top) Coherence of vertical acceleration of free-floating
accelerometer versus shortboard accelerometer near Scripps pier.
(bottom) Phase difference.*